# cuCascade Benchmark Results Visualization

This notebook visualizes benchmark results from Google Benchmark output (gbench_results.json) using Plotly.


In [ ]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path


## Load Benchmark Data


In [ ]:
# Load the JSON file
json_file = 'gbench_results.json'

with open(json_file, 'r') as f:
    data = json.load(f)

# Extract benchmarks
benchmarks = data['benchmarks']

# Convert to DataFrame
df = pd.DataFrame(benchmarks)

print(f"Loaded {len(df)} benchmark results")
print(f"\nColumns: {list(df.columns)}")
df.head()


## Data Preprocessing


In [ ]:
# Extract benchmark name components
def parse_benchmark_name(name):
    """Parse benchmark name to extract test name and parameters."""
    parts = name.split('/')
    base_name = parts[0]
    params = parts[1:] if len(parts) > 1 else []
    return base_name, params

df['base_name'], df['params'] = zip(*df['name'].apply(parse_benchmark_name))

# Calculate throughput in GB/s
if 'bytes_per_second' in df.columns:
    df['throughput_GBs'] = df['bytes_per_second'] / (1024**3)

# Convert time to milliseconds if needed
if 'real_time' in df.columns:
    df['time_ms'] = df['real_time'] if df['time_unit'].iloc[0] == 'ms' else df['real_time'] / 1000

print(f"\nUnique benchmark types: {df['base_name'].unique()}")
df[['name', 'base_name', 'throughput_GBs', 'time_ms']].head(10)


## Throughput Benchmarks Visualization


In [ ]:
# Filter throughput benchmarks
throughput_benchmarks = df[df['base_name'].str.contains('Throughput', na=False)].copy()

if not throughput_benchmarks.empty:
    throughput_benchmarks['size_MB'] = throughput_benchmarks['MB']
    
    # Create throughput comparison plot
    fig = go.Figure()
    
    for bench_name in throughput_benchmarks['base_name'].unique():
        bench_data = throughput_benchmarks[throughput_benchmarks['base_name'] == bench_name].sort_values('size_MB')
        
        fig.add_trace(go.Scatter(
            x=bench_data['size_MB'],
            y=bench_data['throughput_GBs'],
            mode='lines+markers',
            name=bench_name.replace('BM_', '').replace('Throughput', ''),
            line=dict(width=2),
            marker=dict(size=8)
        ))
    
    fig.update_layout(
        title='Memory Transfer Throughput vs Data Size',
        xaxis_title='Data Size (MB)',
        yaxis_title='Throughput (GB/s)',
        xaxis_type='log',
        template='plotly_white',
        hovermode='x unified',
        width=1000,
        height=600,
        font=dict(size=12)
    )
    
    fig.show()
else:
    print("No throughput benchmarks found")


## Facet Plot: Raw Throughput vs Conversions


In [ ]:
# Create a side-by-side comparison of raw throughput vs conversion benchmarks
from plotly.subplots import make_subplots

# Filter for only GpuToHost and HostToGpu
gpu_to_host_pattern = 'GpuToHost'
host_to_gpu_pattern = 'HostToGpu'

filtered_throughput = throughput_benchmarks[
    throughput_benchmarks['base_name'].str.contains(f'{gpu_to_host_pattern}|{host_to_gpu_pattern}', na=False)
].copy()

filtered_conversion = conversion_benchmarks[
    conversion_benchmarks['base_name'].str.contains(f'{gpu_to_host_pattern}|{host_to_gpu_pattern}', na=False)
].copy()

# Create subplot with 1 row and 2 columns with shared y-axis
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Raw Memory Transfer Throughput', 'Conversion Throughput'),
    horizontal_spacing=0.12,
    shared_yaxes=True
)

# Collect all throughput values to determine y-axis range
all_throughputs = []

# Left plot: Raw throughput benchmarks
if not filtered_throughput.empty:
    colors_left = px.colors.qualitative.Set1
    
    for i, bench_name in enumerate(filtered_throughput['base_name'].unique()):
        bench_data = filtered_throughput[filtered_throughput['base_name'] == bench_name].sort_values('size_MB')
        all_throughputs.extend(bench_data['throughput_GBs'].tolist())
        
        fig.add_trace(
            go.Scatter(
                x=bench_data['size_MB'],
                y=bench_data['throughput_GBs'],
                mode='lines+markers',
                name=bench_name.replace('BM_', '').replace('Throughput', ''),
                line=dict(width=2.5),
                marker=dict(size=8),
                legendgroup='throughput',
                legendgrouptitle_text='Raw Transfers',
                showlegend=True
            ),
            row=1, col=1
        )

# Right plot: Conversion benchmarks
if not filtered_conversion.empty:
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 
                      'triangle-down', 'star', 'hexagon', 'pentagon', 'octagon']
    colors_right = px.colors.qualitative.Plotly
    
    # Group by benchmark type
    unique_benchmarks = filtered_conversion['base_name'].unique()
    
    trace_idx = 0
    for bench_idx, bench_name in enumerate(unique_benchmarks):
        bench_data = filtered_conversion[filtered_conversion['base_name'] == bench_name].copy()
        
        if 'size_MB' in bench_data.columns and 'num_columns' in bench_data.columns:
            unique_cols = sorted(bench_data['num_columns'].unique())
            
            for col_idx, num_cols in enumerate(unique_cols):
                col_data = bench_data[bench_data['num_columns'] == num_cols].sort_values('size_MB')
                all_throughputs.extend(col_data['throughput_GBs'].tolist())
                
                # Create a combined name showing both benchmark type and column count
                display_name = f"{bench_name.replace('BM_', '').replace('Convert', '')}"
                trace_name = f"{display_name} ({int(num_cols)}col)"
                
                fig.add_trace(
                    go.Scatter(
                        x=col_data['size_MB'],
                        y=col_data['throughput_GBs'],
                        mode='markers+lines',
                        name=trace_name,
                        marker=dict(
                            size=10,
                            symbol=marker_symbols[col_idx % len(marker_symbols)],
                            color=colors_right[(bench_idx * 3 + col_idx) % len(colors_right)],
                            line=dict(width=1, color='white')
                        ),
                        line=dict(width=1.5),
                        legendgroup='conversion',
                        legendgrouptitle_text='Conversions',
                        showlegend=True
                    ),
                    row=1, col=2
                )
                trace_idx += 1

# Calculate y-axis range with some padding
if all_throughputs:
    y_min = min(all_throughputs) * 0.9
    y_max = max(all_throughputs) * 1.1
else:
    y_min, y_max = 0, 100

# Update axes
fig.update_xaxes(title_text="Data Size (MB)", type="log", row=1, col=1)
fig.update_xaxes(title_text="Data Size (MB)", type="log", row=1, col=2)
fig.update_yaxes(title_text="Throughput (GB/s)", range=[y_min, y_max], row=1, col=1)
fig.update_yaxes(title_text="Throughput (GB/s)", range=[y_min, y_max], row=1, col=2)

# Update layout
fig.update_layout(
    title_text="Performance Comparison: Raw Transfers vs Conversions (GPU↔Host only)",
    template='plotly_white',
    width=1600,
    height=700,
    hovermode='closest',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=1.01,
        font=dict(size=10)
    )
)

fig.show()


## Facet Plot: Transfer Direction vs Benchmark Type


In [ ]:
# Create a 2x2 facet plot: rows for transfer direction, columns for raw/conversion
# Rows: GpuToHost (top), HostToGpu (bottom)
# Columns: Raw (left), Conversion (right)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('GpuToHost Raw', 'GpuToHost Conversion',
                    'HostToGpu Raw', 'HostToGpu Conversion'),
    horizontal_spacing=0.12,
    vertical_spacing=0.15,
    shared_yaxes=True
)

# Collect all throughput values to determine y-axis range
all_throughputs = []

marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 
                  'triangle-down', 'star', 'hexagon', 'pentagon']
colors = px.colors.qualitative.Plotly

# === Top row: GpuToHost ===

# Top-left: GpuToHost Raw
if not filtered_throughput.empty:
    gpu_to_host_raw = filtered_throughput[
        filtered_throughput['base_name'].str.contains('GpuToHost', na=False)
    ].sort_values('size_MB')
    
    if not gpu_to_host_raw.empty:
        all_throughputs.extend(gpu_to_host_raw['throughput_GBs'].tolist())
        
        fig.add_trace(
            go.Scatter(
                x=gpu_to_host_raw['size_MB'],
                y=gpu_to_host_raw['throughput_GBs'],
                mode='lines+markers',
                name='Raw',
                line=dict(width=2.5, color=colors[0]),
                marker=dict(size=8),
                legendgroup='raw',
                showlegend=True
            ),
            row=1, col=1
        )

# Top-right: GpuToHost Conversion
if not filtered_conversion.empty:
    gpu_to_host_conv = filtered_conversion[
        filtered_conversion['base_name'].str.contains('GpuToHost', na=False)
    ].copy()
    
    if not gpu_to_host_conv.empty and 'size_MB' in gpu_to_host_conv.columns:
        unique_cols = sorted(gpu_to_host_conv['num_columns'].unique())
        
        for col_idx, num_cols in enumerate(unique_cols):
            col_data = gpu_to_host_conv[gpu_to_host_conv['num_columns'] == num_cols].sort_values('size_MB')
            all_throughputs.extend(col_data['throughput_GBs'].tolist())
            
            fig.add_trace(
                go.Scatter(
                    x=col_data['size_MB'],
                    y=col_data['throughput_GBs'],
                    mode='markers+lines',
                    name=f'{int(num_cols)} col',
                    marker=dict(
                        size=10,
                        symbol=marker_symbols[col_idx % len(marker_symbols)],
                        color=colors[col_idx % len(colors)],
                        line=dict(width=1, color='white')
                    ),
                    line=dict(width=1.5),
                    legendgroup='conversion',
                    showlegend=True
                ),
                row=1, col=2
            )

# === Bottom row: HostToGpu ===

# Bottom-left: HostToGpu Raw
if not filtered_throughput.empty:
    host_to_gpu_raw = filtered_throughput[
        filtered_throughput['base_name'].str.contains('HostToGpu', na=False)
    ].sort_values('size_MB')
    
    if not host_to_gpu_raw.empty:
        all_throughputs.extend(host_to_gpu_raw['throughput_GBs'].tolist())
        
        fig.add_trace(
            go.Scatter(
                x=host_to_gpu_raw['size_MB'],
                y=host_to_gpu_raw['throughput_GBs'],
                mode='lines+markers',
                name='Raw',
                line=dict(width=2.5, color=colors[0]),
                marker=dict(size=8),
                legendgroup='raw',
                showlegend=False  # Already shown in top-left
            ),
            row=2, col=1
        )

# Bottom-right: HostToGpu Conversion
if not filtered_conversion.empty:
    host_to_gpu_conv = filtered_conversion[
        filtered_conversion['base_name'].str.contains('HostToGpu', na=False)
    ].copy()
    
    if not host_to_gpu_conv.empty and 'size_MB' in host_to_gpu_conv.columns:
        unique_cols = sorted(host_to_gpu_conv['num_columns'].unique())
        
        for col_idx, num_cols in enumerate(unique_cols):
            col_data = host_to_gpu_conv[host_to_gpu_conv['num_columns'] == num_cols].sort_values('size_MB')
            all_throughputs.extend(col_data['throughput_GBs'].tolist())
            
            fig.add_trace(
                go.Scatter(
                    x=col_data['size_MB'],
                    y=col_data['throughput_GBs'],
                    mode='markers+lines',
                    name=f'{int(num_cols)} col',
                    marker=dict(
                        size=10,
                        symbol=marker_symbols[col_idx % len(marker_symbols)],
                        color=colors[col_idx % len(colors)],
                        line=dict(width=1, color='white')
                    ),
                    line=dict(width=1.5),
                    legendgroup='conversion',
                    showlegend=False  # Already shown in top-right
                ),
                row=2, col=2
            )

# Calculate y-axis range with some padding
if all_throughputs:
    y_min = min(all_throughputs) * 0.9
    y_max = max(all_throughputs) * 1.1
else:
    y_min, y_max = 0, 100

# Update axes
for row in [1, 2]:
    for col in [1, 2]:
        fig.update_xaxes(title_text="Data Size (MB)", type="log", row=row, col=col)
        fig.update_yaxes(title_text="Throughput (GB/s)", range=[y_min, y_max], row=row, col=col)

# Update layout
fig.update_layout(
    title_text="Transfer Direction vs Benchmark Type Comparison",
    template='plotly_white',
    width=1400,
    height=900,
    hovermode='closest',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=1.01,
        font=dict(size=10),
        tracegroupgap=20
    )
)

fig.show()
